In [1]:
from util import entropy, information_gain, partition_classes
import numpy as np 
import ast

class DecisionTree(object):
    def __init__(self):
        # Initializing the tree as an empty dictionary or list, as preferred
        #self.tree = []
        self.tree = {}
        pass

    def learn(self, X, y):
        # TODO: Train the decision tree (self.tree) using the the sample X and labels y
        # You will have to make use of the functions in utils.py to train the tree
        
        # One possible way of implementing the tree:
        #    Each node in self.tree could be in the form of a dictionary:
        #       https://docs.python.org/2/library/stdtypes.html#mapping-types-dict
        #    For example, a non-leaf node with two children can have a 'left' key and  a 
        #    'right' key. You can add more keys which might help in classification
        #    (eg. split attribute and split value)

        def split(X, y):
            if len(set(y)) == 1:
                return y[0]
            i = 0 
            abs_max_i = 0
            abs_max_IG = 0
            abs_max_X_left = []
            abs_max_X_right = []
            abs_max_y_left = []
            abs_max_y_right = []
            abs_max_j = 0
            while i < len(X[0]):
                j = 0
                current_H = entropy(y)
                max_j = 0
                max_IG = 0
                max_X_left = []
                max_X_right = []
                max_y_left = []
                max_y_right = []
                while j < len(y):           
                    X_left, X_right, y_left, y_right = partition_classes(X, y, i, X[j][i])
                    if information_gain(y, [y_left, y_right]) > max_IG:
                        max_IG = information_gain(y, [y_left, y_right])
                        max_j = j
                        max_X_left = X_left
                        max_X_right = X_right
                        max_y_left = y_left
                        max_y_right = y_right
                    j += 1
                if max_IG > abs_max_IG:
                    abs_max_IG = max_IG
                    abs_max_i = i
                    abs_max_X_left = max_X_left
                    abs_max_X_right = max_X_right
                    abs_max_y_left = max_y_left
                    abs_max_y_right = max_y_right
                    abs_max_j = max_j
                i += 1
            return {'X_left': abs_max_X_left, 'X_right': abs_max_X_right, 'y_left': abs_max_y_left, 'y_right': abs_max_y_right, 'split_attr': abs_max_i, 'split_val': X[abs_max_j][abs_max_i]}

        def recursive_split(X, y):
            current_dict = split(X,y)
            if (current_dict != 0) and (current_dict != 1):
                current_dict['X_left'] = recursive_split(current_dict['X_left'], current_dict['y_left'])
            if (current_dict != 0) and (current_dict != 1):
                current_dict['X_right'] = recursive_split(current_dict['X_right'], current_dict['y_right'])
            return current_dict

        self.tree = recursive_split(X,y)
        pass


    def classify(self, record):
        # TODO: classify the record using self.tree and return the predicted label
        dict = self.tree
        def route(dict, record):
            if dict == 0 or dict == 1:
                return dict
            if type(record[dict['split_attr']]) == int:
                if record[dict['split_attr']] <= dict['split_val']:
                    return route(dict['X_left'], record)
                else:
                    return route(dict['X_right'], record)
            else:
                if record[dict['split_attr']] == dict['split_val']:
                    return route(dict['X_left'], record)
                else:
                    return route(dict['X_right'], record)

        #def predict(self, record):
        #    predictions = []
         #   for i in record:
          #      predictions.append(route(dict, i))
           # return predictions

        return route(dict, record)
    
        pass


In [2]:
from decision_tree import DecisionTree
import csv
import numpy as np  # http://www.numpy.org
import ast

# This starter code does not run. You will have to add your changes and
# turn in code that runs properly. 

"""
Here, 
1. X is assumed to be a matrix with n rows and d columns where n is the
number of total records and d is the number of features of each record. 
2. y is assumed to be a vector of labels of length n.
3. XX is similar to X, except that XX also contains the data label for each
record.
"""

"""
This skeleton is provided to help you implement the assignment.You must 
implement the existing functions as necessary. You may add new functions
as long as they are called from within the given classes. 

VERY IMPORTANT!
Do NOT change the signature of the given functions.
Do NOT change any part of the main function APART from the forest_size parameter.  
"""

class RandomForest(object):
    num_trees = 0
    decision_trees = []

    # the bootstrapping datasets for trees
    # bootstraps_datasets is a list of lists, where each list in bootstraps_datasets is a bootstrapped dataset.
    bootstraps_datasets = []

    # the true class labels, corresponding to records in the bootstrapping datasets
    # bootstraps_labels is a list of lists, where the 'i'th list contains the labels corresponding to records in 
    # the 'i'th bootstrapped dataset.
    bootstraps_labels = []

    def __init__(self, num_trees):
        # Initialization done here
        self.num_trees = num_trees
        self.decision_trees = [DecisionTree() for i in range(num_trees)]


    def _bootstrapping(self, XX, n):
        # Reference: https://en.wikipedia.org/wiki/Bootstrapping_(statistics)
        #
        # TODO: Create a sample dataset of size n by sampling with replacement
        #       from the original dataset XX.
        # Note that you would also need to record the corresponding class labels
        # for the sampled records for training purposes.

        samples = [] # sampled dataset
        labels = []  # class labels for the sampled records

        r = np.random.randint(n, size = n)
        for i in r:
            rec = XX[i]
            samples.append(rec[0:len(rec)-1])
            labels.append(rec[len(rec)-1])
        
        return (samples, labels)


    def bootstrapping(self, XX):
        # Initializing the bootstap datasets for each tree
        for i in range(self.num_trees):
            data_sample, data_label = self._bootstrapping(XX, len(XX))
            self.bootstraps_datasets.append(data_sample)
            self.bootstraps_labels.append(data_label)


    def fitting(self):
        # TODO: Train `num_trees` decision trees using the bootstraps datasets
        # and labels by calling the learn function from your DecisionTree class.
        for DTree, dataset, label in zip(self.decision_trees, self.bootstraps_datasets, self.bootstraps_labels):
            DTree.learn(dataset, label)
        pass      


    def voting(self, X):
        y = []

        for record in X:
            # Following steps have been performed here:
            #   1. Find the set of trees that consider the record as an 
            #      out-of-bag sample.
            #   2. Predict the label using each of the above found trees.
            #   3. Use majority vote to find the final label for this recod.
            votes = []
            for i in range(len(self.bootstraps_datasets)):
                dataset = self.bootstraps_datasets[i]
                if record not in dataset:
                    OOB_tree = self.decision_trees[i]
                    effective_vote = OOB_tree.classify(record)
                    votes.append(effective_vote)

            counts = np.bincount(votes)
            
            if len(counts) == 0:
                # TODO: Special case 
                #  Handle the case where the record is not an out-of-bag sample
                #  for any of the trees. 
                y = np.append(y, self.decision_trees[0].classify(record))

                pass
            else:
                y = np.append(y, np.argmax(counts))

        return y


In [3]:
X = list()
y = list()
XX = list()  # Contains data features and data labels
numerical_cols = set([1, 2, 7, 10, 13, 14, 15]) # indices of numeric attributes (columns)

# Loading data set
print 'reading hw4-data'
with open("hw4-data.csv") as f:
    next(f, None)

    for line in csv.reader(f, delimiter=","):
        xline = []
        for i in range(len(line)):
            if i in numerical_cols:
                xline.append(ast.literal_eval(line[i]))
            else:
                xline.append(line[i])

        X.append(xline[:-1])
        y.append(xline[-1])
        XX.append(xline[:])

reading hw4-data


In [4]:
    # TODO: Initialize according to your implementation
    # VERY IMPORTANT: Minimum forest_size should be 10
    forest_size = 10
    
    # Initializing a random forest.
    randomForest = RandomForest(forest_size)

    # Creating the bootstrapping datasets
    print 'creating the bootstrap datasets'
    randomForest.bootstrapping(XX)

    # Building trees in the forest
    print 'fitting the forest'
    randomForest.fitting()

    # Calculating an unbiased error estimation of the random forest
    # based on out-of-bag (OOB) error estimate.
    y_predicted = randomForest.voting(X)

    # Comparing predicted and true labels
    results = [prediction == truth for prediction, truth in zip(y_predicted, y)]

    # Accuracy
    accuracy = float(results.count(True)) / float(len(results))

    print "accuracy: %.4f" % accuracy
    print "OOB estimate: %.4f" % (1-accuracy)

creating the bootstrap datasets
fitting the forest
accuracy: 0.8469
OOB estimate: 0.1531
